# Batch Normalization

Batch Normalization é uma técnica que permite normalizar automaticamente os valores que atravessam uma camada da rede neural. 

A normalização é feita para que o resultado tenha média zero e variância unitária, porém, em seguida, o resultado é
escalado pelo fator $\gamma$ e somado ao fator $\beta$. Estes dois fatores são parâmetros que serão também otimizados duranteo treinamento do gradiente descendente.

Atualmente a técnica de batch normalization é utilizada em todas as redes profundas. 

A camada de batch normalization é colocada entre a camada densa ou convolucional e antes da camada de ativação.

## O Algoritmo

Para descrição do algoritmo, veja o artigo original:
- Ioffe, S. & Szegedy, C. (2015), Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift., in Francis R. Bach & David M. Blei, ed., 'ICML' , JMLR.org, , pp. 448-456. [PDF:arxiv](https://arxiv.org/pdf/1502.03167.pdf) 

<img src='../figures/batchnorm_equations.png',width=500></a>

### Equações

\begin{align*} 
\boldsymbol{\mu} &= \frac{1}{m} \sum_{i = 1}^{m} \boldsymbol{x_i} &&
\boldsymbol{\sigma^{2}} = \frac{1}{m} \sum_{i = 1}^{m} (\boldsymbol{x_i} - \boldsymbol{\mu})^{2}
\\[3mm]
\hat{\boldsymbol{x}}_i &= \frac{\boldsymbol{x}_i - \boldsymbol{\mu}}{\sqrt{\boldsymbol{\sigma^{2}} + \epsilon}} &&
\boldsymbol{y_i} = \gamma \hat{\boldsymbol{x}}_i + \beta
\\[3mm]
\boldsymbol{m}_{(t)} &=  \lambda \boldsymbol{m}_{(t-1)} + (1 - \lambda) \boldsymbol{\mu} &&
\boldsymbol{v}_{(t)} =  \lambda \boldsymbol{v}_{(t-1)} + (1 - \lambda) \boldsymbol{\sigma^{2}}
\end{align*}

### Grafo de operações
Figura obtida nesta página: [Understanding the backward pass through Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)

## Importações

In [13]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display

import sys
import numpy as np
import numpy.random as nr
import pandas as pd

import torch
from torch.autograd import Variable

np.set_printoptions(precision=2, suppress=True)
nr.seed(23456)


## Batch Norm: parâmetros

### nn.BatchNorm1d - Entrada com dimensão 2

- Documentação oficial: http://pytorch.org/docs/master/nn.html#normalization-layers

Usualmente, quando a entrada da camada tem duas dimensões (amostras e atributos) a normalização é feita na dimensão dos atributos, *axis=1*. Ou seja, calcula-se a estatística (média e variância) para cada coluna da matriz de dados, em cada *mini-batch*.

In [122]:
torch.manual_seed(1234)
model = torch.nn.BatchNorm1d(5)
x = Variable(torch.randn(4, 5))
model.training = False
y = model(x)

In [123]:
x

Variable containing:
-1.3368 -1.1361  0.2328 -1.7245  0.5111
-0.6126  0.4703  0.6527  0.5719  1.7060
 1.0029 -2.3111  1.1075  1.7168  0.7015
 0.6274  1.1752  0.0955 -1.5116  0.1253
[torch.FloatTensor of size 4x5]

In [124]:
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

weight
[ 0.19  0.5   0.62  0.82  0.44]
bias
[ 0.  0.  0.  0.  0.]
running_mean
[ 0.  0.  0.  0.  0.]
running_var
[ 1.  1.  1.  1.  1.]


In [125]:
y

Variable containing:
-0.2560 -0.5654  0.1449 -1.4103  0.2237
-0.1173  0.2340  0.4061  0.4677  0.7468
 0.1921 -1.1501  0.6890  1.4040  0.3071
 0.1202  0.5849  0.0594 -1.2362  0.0549
[torch.FloatTensor of size 4x5]

## Conferindo

In [126]:
gamma, beta, mv_mean, mv_var = model.state_dict().values()

if model.training:
    y2 = (x - Variable(mv_mean)) / torch.sqrt(Variable(mv_var) + 1e-5)

y3 = y2 * Variable(gamma) + Variable(beta)

print(y3)

Variable containing:
-0.2560 -0.5654  0.1449 -1.4103  0.2237
-0.1173  0.2340  0.4061  0.4677  0.7468
 0.1921 -1.1501  0.6890  1.4040  0.3071
 0.1202  0.5849  0.0594 -1.2362  0.0549
[torch.FloatTensor of size 4x5]



In [127]:
mv_mean.numpy()

array([ 0.,  0.,  0.,  0.,  0.], dtype=float32)

In [5]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(5,), 
                             momentum=0.9, 
                             epsilon=0.0001, 
                             gamma_initializer=Constant(10), 
                             beta_initializer=Constant(11), 
                             moving_mean_initializer=Constant(12), 
                             moving_variance_initializer=Constant(13)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 5)                 20        
Total params: 20
Trainable params: 10
Non-trainable params: 10
_________________________________________________________________


In [3]:
for layer in model.layers:
    print('\nConfiguration:')
    print('--------------')
    for k, v in layer.get_config().items():
        print('  {:30s}: {}'.format(k, v))
    print('\nParameters:')
    print('-----------')
    for p in layer.weights:
        if p in layer.trainable_weights:
            print('  Trainable:', p.name)
        else:
            print('            ', p.name)
print('\nmodel.get_weights():')
print('--------------------')
for w in model.get_weights():
    print('  ', w, w.shape)


Configuration:
--------------
  name                          : batch_normalization_1
  trainable                     : True
  batch_input_shape             : (None, 5)
  dtype                         : float32
  axis                          : 1
  momentum                      : 0.9
  epsilon                       : 0.0001
  center                        : True
  scale                         : True
  beta_initializer              : {'class_name': 'Constant', 'config': {'value': 11}}
  gamma_initializer             : {'class_name': 'Constant', 'config': {'value': 10}}
  moving_mean_initializer       : {'class_name': 'Constant', 'config': {'value': 12}}
  moving_variance_initializer   : {'class_name': 'Constant', 'config': {'value': 13}}
  beta_regularizer              : None
  gamma_regularizer             : None
  beta_constraint               : None
  gamma_constraint              : None

Parameters:
-----------
  Trainable: batch_normalization_1/gamma:0
  Trainable: batch_normaliz

### Entrada com dimensão 4

Usualmente, quando a entrada da camada tem quatro dimensões (amostras, filtros, altura e largura), a normalização é feita na segunda dimensão, *axis=1*, que corresponde aos filtros (canais). Ou seja, calcula-se a estatística (média e variância) para cada *mapa de atributos* do tensor. Assim, preserva-se a propriedade da invariância à translação, característica importante das convoluções.

In [4]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 10, 10), 
                             gamma_initializer=Constant(0), 
                             beta_initializer=Constant(1), 
                             moving_mean_initializer=Constant(2), 
                             moving_variance_initializer=Constant(3)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 3, 10, 10)         12        
Total params: 12
Trainable params: 6
Non-trainable params: 6
_________________________________________________________________


In [5]:
for layer in model.layers:
    print('\nConfiguration:')
    print('--------------')
    for k, v in layer.get_config().items():
        print('  {:30s}: {}'.format(k, v))
    print('\nParameters:')
    print('-----------')
    for p in layer.weights:
        if p in layer.trainable_weights:
            print('  Trainable:', p.name)
        else:
            print('            ', p.name)
print('\nmodel.get_weights():')
print('--------------------')
for w in model.get_weights():
    print('  ', w, w.shape)


Configuration:
--------------
  name                          : batch_normalization_2
  trainable                     : True
  batch_input_shape             : (None, 3, 10, 10)
  dtype                         : float32
  axis                          : 1
  momentum                      : 0.99
  epsilon                       : 0.001
  center                        : True
  scale                         : True
  beta_initializer              : {'class_name': 'Constant', 'config': {'value': 1}}
  gamma_initializer             : {'class_name': 'Constant', 'config': {'value': 0}}
  moving_mean_initializer       : {'class_name': 'Constant', 'config': {'value': 2}}
  moving_variance_initializer   : {'class_name': 'Constant', 'config': {'value': 3}}
  beta_regularizer              : None
  gamma_regularizer             : None
  beta_constraint               : None
  gamma_constraint              : None

Parameters:
-----------
  Trainable: batch_normalization_2/gamma:0
  Trainable: batch_norm

## Keras: Execução na fase de treinamento

In [6]:
K.set_learning_phase(1)

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(5,), momentum=0.5, epsilon=0.0001))

model.set_weights([[ 5., 1., 1., 1., 1.],     # gamma
                   [ 10., 0., 0., 0., 0.],     # beta
                   [ 0., 0., 0., 0., 0.],
                   [ 1., 1., 1., 1., 1.]])

x = nr.random(size=(10, 5)) * 20
print(x, x.mean(0), x.std(0))
print()

y = model.predict(x, batch_size=10)
print(y, y.mean(0), y.std(0))


[[  6.44   6.55  18.55   6.23   3.24]
 [  7.28  10.58  15.78  17.51  12.73]
 [ 19.79  16.32   0.83   4.06   3.05]
 [ 15.06   9.1   19.09  18.75   2.71]
 [  4.72   6.3   13.45  12.41  12.44]
 [ 16.76  12.62  11.22  19.74  13.66]
 [  2.03  18.8   13.03   4.55   8.64]
 [  7.8   15.12   2.85   0.2   16.76]
 [  2.18   4.98   9.52  17.68   3.55]
 [ 15.72   0.7   12.17   6.17  15.4 ]] [  9.78  10.11  11.65  10.73   9.22] [ 6.14  5.37  5.7   6.91  5.35]

[[  7.28  -0.66   1.21  -0.65  -1.12]
 [  7.97   0.09   0.73   0.98   0.66]
 [ 18.16   1.16  -1.9   -0.97  -1.15]
 [ 14.3   -0.19   1.31   1.16  -1.22]
 [  5.88  -0.71   0.32   0.24   0.6 ]
 [ 15.68   0.47  -0.08   1.3    0.83]
 [  3.69   1.62   0.24  -0.89  -0.11]
 [  8.39   0.93  -1.54  -1.52   1.41]
 [  3.81  -0.95  -0.37   1.01  -1.06]
 [ 14.84  -1.75   0.09  -0.66   1.16]] [ 10.   0.   0.  -0.  -0.] [ 5.  1.  1.  1.  1.]


In [7]:
gamma, beta, mv_mean, mv_var = model.get_weights()

x2 = x - x.mean(0)
x2 /= x2.std(0)

x3 = x2 * gamma + beta

print(x3, x3.mean(0), x3.std(0))

[[  7.28  -0.66   1.21  -0.65  -1.12]
 [  7.97   0.09   0.73   0.98   0.66]
 [ 18.16   1.16  -1.9   -0.97  -1.15]
 [ 14.3   -0.19   1.31   1.16  -1.22]
 [  5.88  -0.71   0.32   0.24   0.6 ]
 [ 15.68   0.47  -0.08   1.3    0.83]
 [  3.69   1.62   0.24  -0.89  -0.11]
 [  8.39   0.93  -1.54  -1.52   1.41]
 [  3.81  -0.95  -0.37   1.01  -1.06]
 [ 14.84  -1.75   0.09  -0.66   1.16]] [ 10.  -0.   0.  -0.   0.] [ 5.  1.  1.  1.  1.]


## Keras: Execução na fase de testes

In [14]:
K.set_learning_phase(0)

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(5,), momentum=0.999, epsilon=0.0001))

model.set_weights([[ 1., 1., 1., 1., 1.],     # gamma
                   [ 0., 0., 0., 0., 0.],     # beta
                   [ 0., 0., 0., 0., 0.],     # moving_mean
                   [ 1., 1., 1., 1., 1.]])    # moving_variance

x = nr.random(size=(10, 5)) * 20
print(x, x.mean(0), x.std(0))
print()

y = model.predict(x, batch_size=10)
print(y, y.mean(0), y.std(0))


[[  6.44   6.55  18.55   6.23   3.24]
 [  7.28  10.58  15.78  17.51  12.73]
 [ 19.79  16.32   0.83   4.06   3.05]
 [ 15.06   9.1   19.09  18.75   2.71]
 [  4.72   6.3   13.45  12.41  12.44]
 [ 16.76  12.62  11.22  19.74  13.66]
 [  2.03  18.8   13.03   4.55   8.64]
 [  7.8   15.12   2.85   0.2   16.76]
 [  2.18   4.98   9.52  17.68   3.55]
 [ 15.72   0.7   12.17   6.17  15.4 ]] [  9.78  10.11  11.65  10.73   9.22] [ 6.14  5.37  5.7   6.91  5.35]

[[  6.44   6.55  18.55   6.23   3.24]
 [  7.28  10.58  15.78  17.51  12.73]
 [ 19.79  16.32   0.83   4.06   3.05]
 [ 15.06   9.1   19.09  18.75   2.71]
 [  4.72   6.3   13.45  12.41  12.44]
 [ 16.76  12.62  11.22  19.74  13.66]
 [  2.03  18.8   13.03   4.55   8.63]
 [  7.8   15.11   2.85   0.2   16.76]
 [  2.18   4.98   9.52  17.68   3.55]
 [ 15.72   0.7   12.17   6.17  15.4 ]] [  9.78  10.11  11.65  10.73   9.22] [ 6.14  5.37  5.7   6.91  5.35]


In [15]:
gamma, beta, mv_mean, mv_var = model.get_weights()

x2 = x - mv_mean
x2 /= np.sqrt(mv_var)

x3 = x2 * gamma + beta

print(x3, x3.mean(0), x3.std(0))

[[  6.44   6.55  18.55   6.23   3.24]
 [  7.28  10.58  15.78  17.51  12.73]
 [ 19.79  16.32   0.83   4.06   3.05]
 [ 15.06   9.1   19.09  18.75   2.71]
 [  4.72   6.3   13.45  12.41  12.44]
 [ 16.76  12.62  11.22  19.74  13.66]
 [  2.03  18.8   13.03   4.55   8.64]
 [  7.8   15.12   2.85   0.2   16.76]
 [  2.18   4.98   9.52  17.68   3.55]
 [ 15.72   0.7   12.17   6.17  15.4 ]] [  9.78  10.11  11.65  10.73   9.22] [ 6.14  5.37  5.7   6.91  5.35]


## Keras: Verificando o cálculo das médias móveis

In [59]:
nr.seed(234588)
K.set_learning_phase(1)
mom = 0.2

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(5,), momentum=mom, epsilon=0.0001))
model.add(Dense(1, activation='sigmoid'))

print([x.shape for x in model.get_weights()], end='\n\n')
model.set_weights([[ 1., 1., 1., 1., 1.],     # gamma
                   [ 0., 0., 0., 0., 0.],     # beta
                   [ 0., 0., 0., 0., 0.],     # moving_mean
                   [ 1., 1., 1., 1., 1.]])    # moving_variance

gamma, beta, mv_mean, mv_var, w, b = model.get_weights()

x = nr.random(size=(10, 5)) * 20
y = nr.random(size=(10, 1))
print(x, x.mean(0), x.std(0))
print()

mv_mean_new = (1 - mom) * x.mean(0) + mom * mv_mean
mv_var_new  = (1 - mom) * x.std(0) * x.std(0) + mom * mv_var

sgd = SGD(0.5)
model.compile(sgd, loss='mse')
h = model.fit(x, y, batch_size=10, epochs=1, verbose=0)

new_weights = model.get_weights()

print('lambda:', new_weights[0])
print('beta:  ', new_weights[1])
print()
print('mv_mean:', new_weights[2])
print('        ', mv_mean_new)
print('mv_var: ', new_weights[3])
print('        ', mv_var_new)


[(5,), (5,), (5,), (5,), (5, 1), (1,)]

[[  1.74   4.4    7.11  15.93   7.74]
 [ 15.79  16.52   3.59   8.45   0.57]
 [  6.19   1.2   19.06   9.33   6.74]
 [ 14.86   8.33  19.11  12.7   10.43]
 [  6.52  11.71   0.5   11.32  17.42]
 [  2.81   6.7   15.46  13.86  18.72]
 [ 19.9    5.29  15.4    6.41   5.52]
 [ 15.81   4.68   0.07   0.84  11.68]
 [ 13.75  17.97  17.19   6.23  12.38]
 [  1.05  18.93   1.34   6.03   2.17]] [ 9.84  9.57  9.88  9.11  9.34] [ 6.55  6.    7.67  4.25  5.67]

lambda: [ 1.    0.98  0.98  1.    1.  ]
beta:   [-0.03 -0.02  0.03  0.   -0.02]

mv_mean: [ 7.87  7.66  7.91  7.29  7.47]
         [ 7.87  7.66  7.91  7.29  7.47]
mv_var:  [ 34.54  28.98  47.32  14.66  25.91]
         [ 34.54  28.98  47.32  14.66  25.91]
